In [1]:
import os
import time
import pickle

from collections import defaultdict

import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as ly
sess_opt = tf.ConfigProto(gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.95 , allow_growth=True)
                         ,device_count={'GPU': 1})

from matplotlib import pyplot as plt

from utils import exist_or_mkdir , data_manager , transform_orig

exp_folder = "Attn_ver0"
model_path = "model_para"
tmp_path = "tmp"
log_path = "log"

In [2]:
exp_folder = exist_or_mkdir("./",exp_folder)
model_path = exist_or_mkdir(exp_folder,model_path)
tmp_path = exist_or_mkdir(exp_folder,tmp_path)
log_path = exist_or_mkdir(exp_folder,log_path)

Path : './Attn_ver0'
Path : './Attn_ver0/model_para'
Path : './Attn_ver0/tmp'
Path : './Attn_ver0/log'


## Loading data

In [3]:
Encoder_max_len = 50
Decoder_max_len = 20
min_count = 3

In [4]:
print("### Loading Train Data ###")
data_agent = data_manager("data/all/train.csv" , train=True)

### Loading Train Data ###
Data count : 651339

### Data view ###
Original data  : ['SOS', '短短', '几句', 'EOS', 'v', 'v', 'r', 'uj', 'n', 'NOP', 'ian', 'NOE', '5', 'NOR']
Output Sentence : ['SOS', '像', '看见', '你', '的', '脸', 'EOS']


In [5]:
print("### Loading Test Data ###")
test_agent = data_manager("data/all/test.csv" , train=False)

### Loading Test Data ###
Data count : 70000

### Data view ###
Original data  : ['SOS', '像', '梦', '到', '很', '美的', '歌醒', '了', '却', '不', '记得', 'EOS', 'v', 'v', 'uj', 'r', 'v', 'v', 'd', 'v', 'NOP', 'uo', 'NOE', '8', 'NOR']


## Preprocessing and Padding

In [6]:
idx_in_sen , idx_out_sen , mask_in , mask_out , length_in , idx2word , word2idx , remain_idx = \
    transform_orig([data_agent.orig_data,data_agent.out_sen],min_count=min_count,
                   max_len = [Encoder_max_len,Decoder_max_len])

Min Count : 3
Max Length : [50, 20]
Word Count : 59465
Orig data  : ['SOS', '我', '在', '等', '一分钟', 'EOS', 'd', 'f', 'm', 'NOP', 'ong', 'NOE', '3', 'NOR']
Index data : [36, 24, 199, 1098, 3, 17, 16, 14, 18, 439, 20, 83, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Output Orig data  : ['SOS', '或许', '下', '一分钟', 'EOS']
Output Index data : [2, 2277, 1767, 1098, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
idx_in_sen.shape , idx_out_sen.shape , mask_in.shape , mask_out.shape , length_in.shape

((650611, 50), (650611, 20), (650611, 50), (650611, 20), (650611,))

In [8]:
pickle.dump({"orig_word":[idx2word,word2idx] },
            open(os.path.join(tmp_path,"tokenizer.pkl") , "wb"))

## Build Model

In [9]:
def Encoder(inputs , dim , name , init_state=None , t_len=20 , reuse=False , stack_flag=False):
    cell = tf.contrib.rnn.LSTMCell(dim,name=name,reuse=reuse)
    if init_state:
        state = init_state
    else:
        state = [tf.zeros([tf.shape(inputs)[0] , cell.state_size[0]]),
                 tf.zeros([tf.shape(inputs)[0] , cell.state_size[1]])]
    output_seq = []
    for t in range(t_len):
        if stack_flag:
            out , state = cell(inputs[:,t] , state)
        else:
            out , state = cell(inputs[t] , state)
        output_seq.append(out)
    
    return output_seq , state

In [10]:
def attend_vector(inputs , state , mask , name):
    with tf.name_scope("Attention"):
        state = tf.tile(tf.expand_dims(state , axis=1) , [1,tf.shape(inputs)[1],1])
        concat_vec = tf.concat([inputs,state],axis=-1)
        fc1 = ly.fully_connected(concat_vec,128,activation_fn=tf.nn.leaky_relu,biases_initializer=None,
                                 scope="Attn_{}_1".format(name),reuse=tf.AUTO_REUSE)
        fc2 = ly.fully_connected(fc1,64,activation_fn=tf.nn.leaky_relu,biases_initializer=None,
                                 scope="Attn_{}_2".format(name),reuse=tf.AUTO_REUSE)
        fc3 = ly.fully_connected(fc1,1,activation_fn=None,biases_initializer=None,
                                 scope="Attn_{}_3".format(name),reuse=tf.AUTO_REUSE)
        score = tf.nn.softmax(fc3*mask , axis=1) ## wrong
        ## define my softmax
#         exp_fc3 = tf.exp(fc3)*mask
#         exp_sum = tf.reduce_sum(exp_fc3,axis=1,keepdims=True)
#         score = exp_fc3/exp_sum
    
    return score , tf.reduce_sum(inputs*score , axis=1)

def attn_Encoder(inputs , mask , dim , name , init_state=None , t_len=20 , reuse=False):
    cell = tf.contrib.rnn.LSTMCell(dim,name=name,reuse=reuse)
    if init_state:
        state = init_state
    else:
        state = [tf.zeros([tf.shape(inputs)[0] , cell.state_size[0]]),
                 tf.zeros([tf.shape(inputs)[0] , cell.state_size[1]])]
    output_seq = []
    score_seq = []
    for t in range(t_len):
        score , attn_vec = attend_vector(inputs,state[1],mask,name="Encode")
        out , state = cell(attn_vec,state)
        output_seq.append(out)
        score_seq.append(score)
    
    return output_seq , state , score_seq 


def attn_Decoder(inputs , inputs_E , mask , dim , name , init_state=None , t_len=20 , reuse=False , stack_flag=False):
    cell = tf.contrib.rnn.LSTMCell(dim,name=name,reuse=reuse)
    if init_state:
        state = init_state
    else:
        state = [tf.zeros([tf.shape(inputs)[0] , cell.state_size[0]]),
                 tf.zeros([tf.shape(inputs)[0] , cell.state_size[1]])]
    output_seq = []
    score_seq = []
    for t in range(t_len):
        score , attn_vec = attend_vector(inputs_E,state[1],mask,name="Decode")
        if stack_flag:
            attn_vec = tf.concat([attn_vec,inputs[:,t]] , axis=-1)
        else:
            attn_vec = tf.concat([attn_vec,inputs[t]] , axis=-1)
        out , state = cell(attn_vec,state)
        output_seq.append(out)
        score_seq.append(score)
    
    return output_seq , state , score_seq 


In [11]:
def word_clf(inputs,dim,embd):
    fc1 = ly.fully_connected(inputs,dim,activation_fn=tf.nn.leaky_relu,scope="clf_fc1",reuse=tf.AUTO_REUSE)
    fc2 = ly.fully_connected(fc1,int(embd.shape[0]),activation_fn=None,scope="clf_fc2",reuse=tf.AUTO_REUSE)
    return fc2@embd

In [12]:
def mask_catece(x):
    logit = x[0]
    idx = x[1]
    ce = []
    for t in range(Decoder_max_len-1):
        ce.append( tf.log(tf.nn.embedding_lookup(logit[t],idx[t])+1e-10) )
    return tf.stack(ce)

In [13]:
Seq_g = tf.Graph()
embd_dim = 128

with Seq_g.as_default():
    with tf.name_scope("Input"):
        _in = tf.placeholder(tf.int32,[None,None])
        _in_mask = tf.placeholder(tf.float32,[None,None])
        in_mask = tf.expand_dims(_in_mask,axis=-1)
        
        _in_length = tf.placeholder(tf.int32,[None])
        
        _out = tf.placeholder(tf.int32,[None,Decoder_max_len])
        _out_mask = tf.placeholder(tf.float32,[None,Decoder_max_len])
        gt = _out[:,1::]
        gt_mask = _out_mask[:,1::]
        
        schedual_rate = tf.random_uniform([Decoder_max_len],maxval=1.0)
        schedual_th = tf.placeholder(tf.float32)
        infer_start = tf.ones([tf.shape(_in)[0]],dtype=tf.int32)
        
    with tf.name_scope("Embedding"):
        ## word embedding
        _embd = tf.Variable(tf.truncated_normal([len(idx2word) , embd_dim],stddev=0.1),name="Word_Embd")
        _embd_T = tf.transpose(_embd,[1,0])
        x_vector = tf.nn.embedding_lookup(_embd,_in,max_norm=5)
        y_vector = tf.nn.embedding_lookup(_embd,_out,max_norm=5)
        
    
    
    with tf.name_scope("Encoder"):
        e_cell0 = tf.contrib.rnn.LSTMCell(256,name="E_layer_0",reuse=False)
        e_cell1 = tf.contrib.rnn.LSTMCell(256,name="E_layer_1",reuse=False)
        
        E_layer_0 , E_state_0= tf.nn.dynamic_rnn(e_cell0,x_vector,sequence_length=_in_length,dtype=tf.float32)
        E_layer_1 , E_state_1= tf.nn.dynamic_rnn(e_cell1,E_layer_0,sequence_length=_in_length,dtype=tf.float32)
        
    with tf.name_scope("Decoder"):
        
        D_layer_0 , D_state_0 = Encoder(y_vector,256,"rnn/E_layer_0",init_state=E_state_0,reuse=True,
                                        t_len=Decoder_max_len-1,stack_flag=True)
        D_layer_1 , D_state_1 , D_score = attn_Decoder(D_layer_0,E_layer_1,in_mask,300,name="Attn_D_layer_1",
                                                       init_state=E_state_1,t_len=Decoder_max_len-1,stack_flag=False)
        
        output_seq = []
        for t in range(Decoder_max_len-1):
            choice_input = D_layer_1[t]
            out = word_clf(choice_input,128,_embd_T)
            output_seq.append(out)
        _logits = tf.stack(output_seq,axis=1)
        _prob = tf.nn.softmax(_logits,axis=-1)
        
        
    with tf.name_scope("Loss"):
#         cross_entropy_0 = tf.map_fn(mask_catece,elems=(_prob,gt),dtype=tf.float32)
#         cross_entropy = tf.reduce_sum(cross_entropy_0*gt_mask,axis=-1)/tf.reduce_sum(gt_mask,axis=-1)
#         _loss = -tf.reduce_mean(cross_entropy)

        gt = tf.one_hot(gt,depth=len(idx2word),dtype=tf.float32)
        cross_entropy_0 = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.reshape(gt,[-1,len(idx2word)]),
                                                                     logits=tf.reshape(_logits,[-1,len(idx2word)]))
        cross_entropy_1 = tf.reshape(cross_entropy_0,[-1,Decoder_max_len-1])
        cross_entropy = tf.reduce_sum(cross_entropy_1*gt_mask,axis=-1)/tf.reduce_sum(gt_mask,axis=-1)
        _loss = tf.reduce_mean(cross_entropy)
        
    with tf.name_scope("Train_strategy"):
        opt = tf.train.AdamOptimizer(1e-4)
        _update = opt.minimize(_loss)
    
    with tf.name_scope("Inference"):
        ## start at Encoder layer 2 : E_layer2
        infer_out = tf.nn.embedding_lookup(_embd,infer_start)
        infer_state_0 = E_state_0
        infer_state_1 = E_state_1
        
        infer_score_seq = []
        infer_pred_idx_seq = []
        infer_logits_seq = []
        for t in range(Decoder_max_len-1):
            tmp = Encoder([infer_out],256,"rnn/E_layer_0",init_state=infer_state_0,reuse=True,
                          t_len=1,stack_flag=False)
            infer_layer_0 , infer_state_0 = tmp
            
            tmp = attn_Decoder(infer_layer_0,E_layer_1,in_mask,256,name="Attn_D_layer_1",
                               init_state=infer_state_1,t_len=1,reuse=True,stack_flag=False)
            
            infer_layer_1 , infer_state_1 , infer_score = tmp
            
            infer_score_seq.append(infer_score)
            
            infer_out = word_clf(infer_layer_1[0],128,_embd_T)
            infer_logits_seq.append(infer_out)
            
            out_index = tf.argmax(infer_out,axis=1)
            infer_pred_idx_seq.append(out_index)
            infer_out = tf.nn.embedding_lookup(_embd , out_index)
            
        infer_pred_idx_seq = tf.stack(infer_pred_idx_seq,axis=1)
        infer_logits = tf.stack(infer_logits_seq,axis=1)
        infer_prob = tf.nn.softmax(infer_logits,axis=-1)
    
    all_var = tf.trainable_variables()
#     tf.summary.FileWriter(log_path,graph=Seq_g)
    _init = tf.global_variables_initializer()
    saver = tf.train.Saver(max_to_keep=10,var_list=tf.global_variables())
    
print("Finish Building!!\n")

ValueError: Dimensions must be equal, but are 768 and 812 for 'Decoder/Attn_D_layer_1/MatMul' (op: 'MatMul') with input shapes: [?,768], [812,1200].

In [ ]:
def calculate_var(var):
    d = 0
    for w in var:
        tmp_d = 1
        for ww in w.shape:
            tmp_d *= int(ww)
        d += tmp_d
    print("Var Count :" , d)
    return d

## Training

In [14]:
sess = tf.Session(graph=Seq_g,config=sess_opt)
sess.run(_init)

In [15]:
def get_batch(i):
    tmp_end = max(length_in[i])
    my_dict = {
        _in:idx_in_sen[i,:tmp_end],
        _in_mask:mask_in[i,:tmp_end],
        _out:idx_out_sen[i],
        _out_mask:mask_out[i],
        _in_length:length_in[i]
    }
    return my_dict

In [16]:
def evaluate_batch(sess,_pred,count=3):
    idx = np.random.choice(idx_in_sen.shape[0],[count])
    tmp_max_len = max(length_in[idx])
    my_dict = {
        _in:idx_in_sen[idx,:tmp_max_len],
        _in_mask:mask_in[idx,:tmp_max_len],
        _in_length:length_in[idx]
    }
    pred = sess.run(_pred , feed_dict=my_dict)
    
    word_seq = []
    for i in range(3):
        idx_sen = pred[i]
        tmp = []
        for t in range(Decoder_max_len-1):
            if(idx_sen[t] == 3):
                break
            tmp.append(idx2word[idx_sen[t]])
        word_seq.append(tmp)
    
    print("Max length :" , tmp_max_len)
    for i in range(3):
        print("  Input word  :" , data_agent.orig_data[remain_idx[idx[i]]])
        print("  Input index :" , idx_in_sen[idx[i],:tmp_max_len])
        print("  Ground word :" , data_agent.out_sen[remain_idx[idx[i]]])
        print("    Output    :" , word_seq[i])
        print()

In [18]:
batch_size = 300
n_epoch = 100
n_step = idx_in_sen.shape[0]//batch_size

r_index = np.arange(idx_in_sen.shape[0])
loss_list = []

for e in range(1,n_epoch+1):
    np.random.shuffle(r_index)
    start_time = time.time()
    start = 0
    for s in range(n_step):
        idx = r_index[start:start+batch_size]
        _,l = sess.run([_update,_loss] , feed_dict=get_batch(idx))
        start += batch_size
        print("step {:>5d} loss : {:>9.4f} time : {:>7.2f}".format(s,l,time.time()-start_time) , end="\r")
        if s % 500 == 0:
            print("step {:>5d} loss : {:>9.4f} time : {:>7.2f}".format(s,l,time.time()-start_time) , end="\n")
            evaluate_batch(sess,infer_pred_idx_seq,3)
    
    loss_list.append(l)
    print("\nEpoch {0:>3d}/{1:d} loss : {2:>9.4f} time : {3:>8.2f}".format(e,n_epoch,l,time.time()-start_time))
    
    evaluate_batch(sess,infer_pred_idx_seq,3)
    
    if e%4 == 0:
        saver.save(sess,os.path.join(model_path,"model_{}.ckpt".format(e)))
        pickle.dump(loss_list,open(os.path.join(log_path,"loss.pkl") , "wb"))


step     0 loss :    6.4901 time :    0.37
Max length : 25
  Input word  : ['SOS', '抵挡', '热情', '抵挡', '冷落', '抵挡', '太', '慷慨', 'EOS', 'v', 'v', 'v', 's', 'n', 'uj', 'v', 'NOP', 'i', 'NOE', '7', 'NOR']
  Input index : [6413  727 6413 6604 6413  291 3813    3   15   15   15   72   31   41
   15   18   32   20   66   22    0    0    0    0    0]
  Ground word : ['SOS', '停下', '呼吸', '静听', '身边', '低声', '的', '哭泣', 'EOS']
    Output    : ['我', '你', '的', '的', '的', '的']

  Input word  : ['SOS', '等待', '我', '在', '等', '着', '你', '回来', '耶', 'EOS', 'nz', 'r', 'p', 'r', 'uj', 'n', 'r', 'r', 'd', 'p', 'v', 'NOP', 'uai', 'NOE', '11', 'NOR']
  Input index : [ 980   36   24  199   44    5 1589 3635    3  626   12   13   12   41
   31   12   12   17   13   15   18  332   20  248   22]
  Ground word : ['SOS', '佳佳', '我', '给', '你', '的', '爱', '为何', '你', '还', '在', '徘徊', 'EOS']
    Output    : ['我', '我', '你', '的', '的', '的', '的', '的', '的']

  Input word  : ['SOS', '你', '扔下', '手中', '的', '道具', 'EOS', 'v', 'v', 'mq', 'n'

step  1500 loss :    5.1673 time :  515.28
Max length : 14
  Input word  : ['SOS', '向着', '梦', '方向', '奔跑', 'EOS', 'l', 'm', 'uj', 'n', 'NOP', 'ang', 'NOE', '4', 'NOR']
  Input index : [10201   693   845   602     3    63    14    41    31    18   142    20
    73    22]
  Ground word : ['SOS', '一路上', '点点滴滴', '的', '伤', 'EOS']
    Output    : ['OOV', '人', '的', '时候']

  Input word  : ['SOS', '最美', '的', '澎湃', 'EOS', 'r', 'p', 'uj', 'n', 'NOP', 'ai', 'NOE', '4', 'NOR']
  Input index : [1335   10 6642    3   12   13   41   31   18  206   20   73   22    0]
  Ground word : ['SOS', '你', '给', '的', '爱', 'EOS']
    Output    : ['我', '在', '的', '时候']

  Input word  : ['SOS', '笑一笑', '跳', '一', '跳', 'EOS', 'a', 'n', 'a', 'a', 'NOP', 'ao', 'NOE', '4', 'NOR']
  Input index : [4106 2094 1477 2094    3   40   31   40   40   18 1224   20   73   22]
  Ground word : ['SOS', '快乐', '女孩', '乐', '逍遥', 'EOS']
    Output    : ['啊', '里', '好', '好']

step  1625 loss :    5.1481 time :  558.20
Epoch   2/100 loss :    5.

step   500 loss :    4.7185 time :  171.82
Max length : 17
  Input word  : ['SOS', '某', '天', '醒来', '我会', '记得', 'EOS', 'v', 'v', 'n', 'n', 'uj', 'nz', 'NOP', 'ong', 'NOE', '6', 'NOR']
  Input index : [4425  647 1577  961 2715    3   15   15   31   31   41  626   18  439
   20   33   22]
  Ground word : ['SOS', '记得', '相遇', '时', '嘴角', '的', '彩虹', 'EOS']
    Output    : ['是', '是', 'OOV', '的', 'OOV', 'OOV']

  Input word  : ['SOS', '若', '是', '无缘', '再见', 'EOS', 'nr', 'nr', 'v', 'm', 'NOP', 'ian', 'NOE', '4', 'NOR']
  Input index : [ 261   96 3534 1501    3   39   39   15   14   18  124   20   73   22
    0    0    0]
  Ground word : ['SOS', '白堤', '柳帘', '垂泪', '好几遍', 'EOS']
    Output    : ['OOV', 'OOV', '地', 'OOV']

  Input word  : ['SOS', '感到', '正', '慢慢', '丢失', '着', '青春', 'EOS', 'd', 'n', 'v', 'NOP', 'ui', 'NOE', '3', 'NOR']
  Input index : [2081 1593  923 2613   44 2592    3   17   31   15   18  279   20   83
   22    0    0]
  Ground word : ['SOS', '都', '无法', '追回', 'EOS']
    Output    : ['

step  1625 loss :    4.0690 time :  558.88
Epoch   5/100 loss :    4.0690 time :   558.88
Max length : 23
  Input word  : ['SOS', '如', '像', '没有', '旋律', '的', '情歌', 'EOS', 'r', 'v', 'l', 'r', 'd', 'v', 'a', 'NOP', 'u', 'NOE', '7', 'NOR']
  Input index : [ 321  178  218 4354   10 6332    3   12   15   63   12   17   15   40
   18   19   20   66   22    0    0    0    0]
  Ground word : ['SOS', '我', '说', '对不起', '你', '却', '记不起', '苦楚', 'EOS']
    Output    : ['我', '是', '我', '爱', '别', '很', '好']

  Input word  : ['SOS', '伤', '了', '心', '有些', '事', '也', '要', '过去', 'EOS', 'n', 'd', 'a', 'uj', 'v', 'd', 'v', 'r', 'r', 'NOP', 'i', 'NOE', '9', 'NOR']
  Input index : [1984   52  608  661 1062   68  374 3411    3   31   17   40   41   15
   17   15   12   12   18   32   20   21   22]
  Ground word : ['SOS', '心', '很', '痛痛', '的', '不想', '再', '做', '我', '自己', 'EOS']
    Output    : ['OOV', '太', '好', '的', '爱', '都', '是', '你', '你']

  Input word  : ['SOS', '请', '你', '倾听', '我们', '诚恳', '真挚', '的', '心情', 'EOS', 'v

step   500 loss :    3.8355 time :  173.04
Max length : 23
  Input word  : ['SOS', '虽然', '你', '不会', '表达', '你', '的', '真情', 'EOS', 'd', 'v', 'ul', 'n', 'uj', 'vn', 'NOP', 'ing', 'NOE', '6', 'NOR']
  Input index : [2132    5  102 5251    5   10 6747    3   17   15   50   31   41   93
   18   65   20   33   22    0    0    0    0]
  Ground word : ['SOS', '却', '付出', '了', '热忱', '的', '生命', 'EOS']
    Output    : ['就', '是', '了', '爱情', '的', '旅行']

  Input word  : ['SOS', '完美', '的', '关系', '是', '最', '难练', '的', '表演', 'EOS', 'a', 'v', 'i', 'c', 'v', 'r', 'n', 'r', 'v', 'NOP', 'uan', 'NOE', '9', 'NOR']
  Input index : [  229    10  4477    96    57 58278    10  1367     3    40    15    32
    64    15    12    31    12    15    18   194    20    21    22]
  Ground word : ['SOS', '幸福', '是', '一厢情愿', '还是', '应该', '你', '情', '我', '愿', 'EOS']
    Output    : ['快', '是', 'OOV', '和', '让', '我', '的', '是', '拥抱']

  Input word  : ['SOS', '实在', '是', '应当', '寒心', '的', 'EOS', 'c', 'v', 'n', 'uj', 'n', 'NOP', 'iang',

...

step  1625 loss :    1.9660 time :  561.22
Epoch  93/100 loss :    1.9660 time :   561.22
Max length : 23
  Input word  : ['SOS', '开始', '闹', '得', '还', '不', '大声', 'EOS', 't', 'l', 'NOP', 'eng', 'NOE', '2', 'NOR']
  Input index : [1086 5467  516    6  270  954    3   92   63   18  375   20  126   22
    0    0    0    0    0    0    0    0    0]
  Ground word : ['SOS', '后来', '借酒装疯', 'EOS']
    Output    : ['现在', '一阵风']

  Input word  : ['SOS', '哥哥', '你', '别忘了', '我', '呀', 'EOS', 'r', 'v', 'r', 'a', 'uj', 'n', 'NOP', 'iang', 'NOE', '6', 'NOR']
  Input index : [  889     5 11723    36   681     3    12    15    12    40    41    31
    18   190    20    33    22     0     0     0     0     0     0]
  Ground word : ['SOS', '我', '是', '你', '亲爱', '的', '梅娘', 'EOS']
    Output    : ['你', '是', '我', '亲爱', '的', '梅娘']

  Input word  : ['SOS', '合如真如', '假如', '可', '分身', '饰演', '自己', 'EOS', 'v', 'd', 's', 'uj', 'a', 'v', 'p', 'r', 'b', 'uj', 'n', 'NOP', 'i', 'NOE', '11', 'NOR']
  Input index : [42138  144

step   500 loss :    1.9469 time :  172.68
Max length : 24
  Input word  : ['SOS', '跑马', '溜溜', '的', '山上', '一朵', '溜溜', '的', '云', '哟', 'EOS', 'n', 'z', 'uj', 'v', 'n', 'z', 'uj', 'n', 'y', 'NOP', 'o', 'NOE', '9', 'NOR']
  Input index : [26838 14285    10  3475  1050 14285    10  1874  2420     3    31   480
    41    15    31   480    41    31   563    18   150    20    21    22]
  Ground word : ['SOS', '端端', '溜溜', '的', '照在', '康定', '溜溜', '的', '城', '哟', 'EOS']
    Output    : ['端端', '溜溜', '的', '照在', '康定', '溜溜', '的', '姑娘', '哟']

  Input word  : ['SOS', '漫天', '的', '星光', 'EOS', 'n', 'v', 'n', 'NOP', 'ing', 'NOE', '3', 'NOR']
  Input index : [1042   10 2737    3   31   15   31   18   65   20   83   22    0    0
    0    0    0    0    0    0    0    0    0    0]
  Ground word : ['SOS', '天使', '掠过', '头顶', 'EOS']
    Output    : ['泪', '烧成', '光明']

  Input word  : ['SOS', '无数个', '最', '坏', '时分', 'EOS', 'b', 's', 'uj', 'r', 'p', 'r', 'd', 'nr', 'NOP', 'un', 'NOE', '8', 'NOR']
  Input index : [22343

step  1500 loss :    1.8315 time :  518.12
Max length : 19
  Input word  : ['SOS', '有', '多少', '次', '我', '尝试', '写', '首诗', 'EOS', 'v', 'r', 'r', 'i', 'uj', 'n', 'NOP', 'i', 'NOE', '6', 'NOR']
  Input index : [  318    61  3446    36  9801  1351 22150     3    15    12    12    32
    41    31    18    32    20    33    22]
  Ground word : ['SOS', '留下', '你', '那', '婀娜多姿', '的', '样子', 'EOS']
    Output    : ['属于', '你', '我', '无坚不摧', '的', '记忆']

  Input word  : ['SOS', '在', '你', '和', '我', '的', '心里', 'EOS', 'r', 'p', 'n', 't', 'NOP', 'u', 'NOE', '4', 'NOR']
  Input index : [ 24   5 385  36  10 253   3  12  13  31  92  18  19  20  73  22   0   0
   0]
  Ground word : ['SOS', '我们', '在', '用心', '过不去', 'EOS']
    Output    : ['我', '在', '人海', '下去']

  Input word  : ['SOS', '怕', '什么', '我', '坚拒', '为', '情敌', '驳火', 'EOS', 'n', 'd', 'v', 'NOP', 'o', 'NOE', '3', 'NOR']
  Input index : [  139   135    36 30300   152   953 45887     3    31    17    15    18
   150    20    83    22     0     0     0]
  Grou

step   500 loss :    1.8242 time :  172.23
Max length : 21
  Input word  : ['SOS', '让', '烦恼', '都', '灰飞', 'EOS', 'd', 'v', 'n', 'r', 'vg', 'vg', 'NOP', 'ei', 'NOE', '6', 'NOR']
  Input index : [   84  2340   187 15055     3    17    15    31    12   272   272    18
   186    20    33    22     0     0     0     0     0]
  Ground word : ['SOS', '别', '去', '理会', '自我', '藉', '慰', 'EOS']
    Output    : ['别', '去', '理会', '自我', '藉', '慰']

  Input word  : ['SOS', '这', '梦', '人', '寻觅', '爱心', '默然', '无', '语', 'EOS', 'r', 'n', 'n', 'v', 'ud', 'n', 'n', 'NOP', 'in', 'NOE', '7', 'NOR']
  Input index : [   95   693   454  7165  4109 16082   837  6171     3    12    31    31
    15   514    31    31    18    86    20    66    22]
  Ground word : ['SOS', '这', '梦', '人', '盼', '得', '君', '亲亲', 'EOS']
    Output    : ['谁', '人', '心', '爱', '得', '像片', '灰烬']

  Input word  : ['SOS', '皓月', '满', '空', 'EOS', 'd', 'v', 'n', 'NOP', 'ei', 'NOE', '3', 'NOR']
  Input index : [45309  2922   688     3    17    15    31    1

step  1625 loss :    1.7580 time :  560.71
Epoch  99/100 loss :    1.7580 time :   560.71
Max length : 27
  Input word  : ['SOS', '把', '爱', 'EOS', 'd', 'NOP', 'uan', 'NOE', '1', 'NOR']
  Input index : [566  54   3  17  18 194  20 122  22   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0]
  Ground word : ['SOS', '永远', 'EOS']
    Output    : ['永远']

  Input word  : ['SOS', '听见', '这', '碍', '耳', '声线', '浮夸', '的', '四个', '懒惰', '汉', 'EOS', 'v', 'p', 'n', 'v', 'v', 'p', 'r', 'v', 'r', 'v', 'ug', 'NOP', 'uo', 'NOE', '11', 'NOR']
  Input index : [  618    95 55731 10535  3365 11148    10 23892  2169  5558     3    15
    13    31    15    15    13    12    15    12    15   452    18   461
    20   248    22]
  Ground word : ['SOS', '不想', '把', '耳朵', '遮掩', '不想', '给', '你', '发觉', '我', '听', '过', 'EOS']
    Output    : ['OOV', '把', 'OOV', 'OOV', 'OOV', '把', '我', '使', '你', 'OOV', '过']

  Input word  : ['SOS', '你', '的', '笑容', '这样', '熟悉', 'EOS', 'r', 'd', 'v', 'NOP', 'i', 'NOE', '3'

## Inference

In [23]:
idx2word[0:4]

['NULL', 'OOV', 'SOS', 'EOS']

In [39]:
def infer_test_data(sess,_pred,x,word2idx,batch_size=1000):
    start = 0
    pred_word_seq = []
    while(start < len(x)):
        batch_idx_sen = []
        batch_length = []
        batch_mask = []
        max_len = 0
        for s in x[start : start+batch_size]:
            l = len(s)-1
            if(l>max_len):
                max_len = l
        
        for s in x[start : start+batch_size]:
            arr = []
            ## s[1::] : remove first word "SOS"
            batch_mask.append( np.zeros([max_len]))
            batch_mask[-1][0:len(s[1::])] += 1 
            batch_length.append(len(s[1::]))
            for ss in s[1::]:
                try:
                    arr.append(word2idx[ss])
                except:
                    arr.append(1)
            arr.extend([0]*(max_len-len(s[1::])))
            batch_idx_sen.append(arr)
        batch_idx_sen = np.array(batch_idx_sen)
        batch_length = np.array(batch_length)
        batch_mask = np.stack(batch_mask)
        
        pred_sen = sess.run(_pred,feed_dict={
            _in:batch_idx_sen,
            _in_length:batch_length,
            _in_mask:batch_mask
        })
        
        for i in range(batch_size):
            idx_sen = pred_sen[i]
            tmp = []
            for t in range(Decoder_max_len-1):
                if(idx_sen[t] == 3):
                    break
                elif(idx_sen[t] == 1):
                    tmp.append(np.random.choice(idx2word))
                else:
                    tmp.append(idx2word[idx_sen[t]])
            pred_word_seq.append(" ".join(tmp))
        start += batch_size
        
    return pred_word_seq

In [40]:
test_infer = infer_test_data(sess,infer_pred_idx_seq,test_agent.orig_data,word2idx,batch_size=1000)

In [46]:
for i in np.random.choice(len(test_agent.orig_data) , 10 , replace=False):
    print("Input : " , " ".join(test_agent.orig_data[i]))
    print("Infer : " , test_infer[i])
    print()

Input :  SOS 我 看见 快乐 在 对 我 笑 EOS r v a p v NOP iao NOE 5 NOR
Infer :  我 忍住 沉默 在 咆哮

Input :  SOS 这 说明 EOS n n uv v NOP ei NOE 4 NOR
Infer :  眼光 无力 地 凋萎

Input :  SOS 雨 不再 落下 EOS d r m n NOP ang NOE 4 NOR
Infer :  就 这样 一点 信仰

Input :  SOS 野中 EOS zg NOP ong NOE 1 NOR
Infer :  有恃无恐

Input :  SOS 凝固 发慌 的 时光 EOS c r d v n p NOP ai NOE 6 NOR
Infer :  而 我 已 没有 一遭 在

Input :  SOS 不让 我 再 为情所伤 EOS r v v p n f v NOP uo NOE 7 NOR
Infer :  我 想 站 在 梦 里 坠落

Input :  SOS 那些 路人甲 们 凭 什么 发言 惹人讨厌 准备 惊艳 EOS r c v n n uj nr NOP ie NOE 7 NOR
Infer :  他们 无为 米粮川 艾伦 线装书 的 纯氧

Input :  SOS 花若离 枝 随 莲 去 EOS v d v c NOP i NOE 4 NOR
Infer :  爱 不 舍 所以

Input :  SOS 算式 多 高深 多 艰涩 也 不想 停 EOS v v v r v d m a NOP ing NOE 8 NOR
Infer :  可 能 让 我 有 太 多 平静

Input :  SOS 叫 我 继续 追寻 EOS r uj i NOP ong NOE 3 NOR
Infer :  一样 的 海阔天空



In [47]:
test_infer[i]

'一样 的 海阔天空'

In [53]:
def save_infer(data,name):
    path = os.path.join(exp_folder,name)
    print("Save at '{}'".format( path))
    with open( path, "w") as f:
        for s in data:
            s = "".join(s.split())
            if(len(s) == 0):
                s = np.random.choice(idx2word[4::])
            f.write(s+"\n")

def save_infer_seg(data,name):
    path = os.path.join(exp_folder,name)
    print("Save at '{}'".format( path))
    with open( path, "w") as f:
        for s in data:
            if(len(s) == 0):
                s = np.random.choice(idx2word[4::])
            f.write(s+"\n")


In [54]:
save_infer(test_infer,"infer_output.txt")
save_infer_seg(test_infer,"infer_seg.txt")

Save at './Orig_data_ver0/infer_output.txt'
Save at './Orig_data_ver0/infer_seg.txt'
